# Monetary and Fiscal Policy Coordination Notebook


In [75]:
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.tsa.seasonal as tsa
import statsmodels.api as sm
import datetime as dt
import numpy as np

from lets_plot import *
LetsPlot.setup_html()
from IPython.display import Image
import cairosvg

### Functions I made to make datetime conversion easier.

In [76]:
def strpquarter(i):
    i = str(i)
    i = i[0:4] + '-' + i[4:6]
    return pd.to_datetime(i)

In [77]:
def strptimer(i):
    i = str(i)
    i = i[0:4] + '-' + i[4:6]
    return pd.to_datetime(i)

## Taylor Rule Deviation:

$i_t = 2 + \pi_t + a(\pi_t-\pi^*)+b(y_t - y_t^*)$

+ From the formula above we need:

    + Interest Rate
    + Inflation Rate
    + Targetted Inflation Rate
    + Potential Output

### Interest Rates:

Sources: https://iftp.chinamoney.com.cn/english/bmklpr/, https://www.bis.org/statistics/cbpol/cbpol.xlsx

I need to quarterlyize the interest rates

In [78]:
# Static Variables

pstar = 0.03
ustar = 0.0486

# Getting my dataset for interest rates

df = pd.DataFrame()
for i in range(0,8): # For between 2013-2020.
    t = pd.read_excel(f'../data/china/interest-rates/Loan_Prime_Rate_Historical_Data ({i}).xlsx',engine='openpyxl')
    t = t.loc[~((t['Date']=='Data source:')|(t['Date'].isnull())),:]
    df = pd.concat([df,t], ignore_index=True)

# Getting the dataset for benchmark rates 2010-2013.
t = pd.read_excel('../data/china/interest-rates/cbpol.xlsx', engine = 'openpyxl', sheet_name = 'Monthly Series')
t.columns = t.loc[2]
t = t.drop([0,1,2])
t = t.loc[(~(t['M:CN'].isnull())&(t['Period']<dt.datetime(2013,10,25))&(t['Period']>dt.datetime(2009,12,31))),['Period','M:CN']]
t['Date'] = t['Period']
t['1Y'] = t['M:CN']
df = pd.concat([df,t], ignore_index=True)

# Cleaning the data up.
df['Date'] = pd.to_datetime(df['Date'])
df['1Y'] = df['1Y'].astype(float)/100
df = df.rename(columns={'1Y':'OneYearLPRRate'})
df = df.sort_values(by='Date',axis=0)
df = df.reset_index()
df = df.drop(columns =['Period', 'M:CN','5Y', 'index'])

# Changing to quarterly data
df['Quarter'] = pd.PeriodIndex(df['Date'],freq = 'Q')
df = df.groupby('Quarter', as_index=False).mean()
df['QuarterDate'] = df['Quarter'].apply(strpquarter)
df = df.set_index(df['QuarterDate'])
df = df.drop(columns = ['Date', 'QuarterDate'])

# Inflation Stat
df1 = pd.read_csv('../data/china/inflation-rates/CPALTT01CNQ659N.csv')
df1['observation_date'] = pd.to_datetime(df1['observation_date'])
df1 = df1.set_index(df1['observation_date'])
df1 = df1.drop(columns=['observation_date'])
df1 = df1['2010-01-01':'2020-12-31']
df['Inflation'] = df1['CPALTT01CNQ659N']/100

# GDP Stat
df1 = pd.read_csv('../data/china/real-GDP/CHNGDPNQDSMEI.csv')
df1['observation_date'] = pd.to_datetime(df1['observation_date'])
df1 = df1.set_index(df1['observation_date'])
df1 = df1.drop(columns=['observation_date'])
df1 = df1['2010-01-01':'2020-12-31']
df['NominalGDP'] = df1['CHNGDPNQDSMEI']
df['NominalGDP']=tsa.STL(df['NominalGDP']).fit().trend

# Unemployment Stat
df1 = pd.read_excel('../data/china/unemployment-rates/imf-dm-export-modified-20250226.xls')
df1 = df1.transpose()
df1 = df1.reset_index()
df1.columns = df1.loc[0]
df1 = df1.rename(columns={'Unemployment Rate':'UnemploymentRate'})
df1 = df1.drop([0])
df1['Year'] = pd.to_datetime(df1['Year'], format='%Y')
df1 = df1.set_index(df1['Year'])
df1 = df1.drop(columns=['Year'])
df['UnemploymentRate'] = df1['UnemploymentRate']/100
df['UnemploymentRate'] = df['UnemploymentRate'].fillna(method='ffill')

# Ystar
df['Ystar'] = df['NominalGDP']/(1-0.4*(df['UnemploymentRate']-ustar))
#gdp_trend, gdp_cycle = sm.tsa.filters.hpfilter(df["NominalGDP"], lamb=1600)
#output_gap = (df["NominalGDP"] - gdp_trend) / gdp_trend * 100
#df['Ystar'] = df['NominalGDP'] + output_gap
#df["OutputGap"] = output_gap

# istar
df['OutputGap'] = (df['NominalGDP']-df['Ystar'])/df['Ystar']
df['Istar'] = 2 + 1.5*df['Inflation'] + 0.5*df['OutputGap']+1

# Taylor Rule Deviations
df['TaylorDeviation'] = (df['OneYearLPRRate']-df['Istar'])/100


# Debt and Primary Expenditure for Fiscal Policy Response Indicator
df1 = pd.read_excel('../data/china/government-spending/imf-dm-export-20250226 (1).xls')
df1 = df1.transpose()
df1 = df1.reset_index()
df1.columns = df1.loc[0]
df1 = df1.rename(columns = {'Government primary expenditure, percent of GDP (% of GDP)':'PrimExp',
                            'General government gross debt (Percent of GDP)':'Debt'})
df1 = df1.drop([0])
df1['Year'] = pd.to_datetime(df1['Year'], format = '%Y')
df1 = df1.set_index('Year')

df['Debt'] = -df1['Debt']/100
df['DebtMinusHalf'] = df['Debt'].shift(2)
df['PrimExp'] = -df1['PrimExp']/100
df['Debt'] = df['Debt'].fillna(method='ffill')
df['PrimExp'] = df['PrimExp'].fillna(method='ffill')
df['DebtMinusHalf'] = df['DebtMinusHalf'].fillna(method='ffill')
df.loc['2010-01-01':'2010-04-01','DebtMinusHalf'] = df.loc['2010-07-01','DebtMinusHalf']

df.to_csv('../data/china/chinese_coordinator.csv')
df

c:\Users\ellem\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ellem\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ellem\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ellem\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl'

,Quarter,OneYearLPRRate,Inflation,NominalGDP,UnemploymentRate,Ystar,OutputGap,Istar,TaylorDeviation,Debt,DebtMinusHalf,PrimExp
QuarterDate,,,,,,,,,,,,
2010-01-01,2010Q1,0.053100,0.020716,9.556961e+12,0.0410,9.527995e+12,0.00304,3.032593,-0.029795,-0.339,-0.339,-0.246429
2010-04-01,2010Q2,0.053100,0.027181,1.003549e+13,0.0410,1.000507e+13,0.00304,3.042291,-0.029892,-0.339,-0.339,-0.246429
2010-07-01,2010Q3,0.053100,0.032646,1.051467e+13,0.0410,1.048281e+13,0.00304,3.050488,-0.029974,-0.339,-0.339,-0.246429
2010-10-01,2010Q4,0.056433,0.046423,1.099267e+13,0.0410,1.095936e+13,0.00304,3.071155,-0.030147,-0.339,-0.339,-0.246429
2011-01-01,2011Q1,0.059767,0.052236,1.147361e+13,0.0410,1.143883e+13,0.00304,3.079875,-0.030201,-0.338,-0.339,-0.266266
2011-04-01,2011Q2,0.063100,0.059541,1.192189e+13,0.0410,1.188576e+13,0.00304,3.090831,-0.030277,-0.338,-0.339,-0.266266
2011-07-01,2011Q3,0.065600,0.064384,1.231944e+13,0.0410,1.228211e+13,0.00304,3.098097,-0.030325,-0.338,-0.338,-0.266266
2011-10-01,2011Q4,0.065600,0.046209,1.267303e+13,0.0410,1.263462e+13,0.00304,3.070834,-0.030052,-0.338,-0.338,-0.266266
2012-01-01,2012Q1,0.065600,0.037273,1.298816e+13,0.0410,1.294879e+13,0.00304,3.057429,-0.029918,-0.344,-0.338,-0.276698


In [79]:
plot = df.copy()
plot['Ystar'] = plot['NominalGDP']/((1-0.4*(df['UnemploymentRate']-ustar)))
plot3 = (
    ggplot(data = plot.reset_index(), mapping = aes(x = 'QuarterDate'))
    + geom_line(aes(y = 'NominalGDP'), manual_key = 'GDP', color = 'red')
    + geom_line(aes(y = 'Ystar'), manual_key = 'Potential GDP', color = 'black')
    + theme(legend_position = 'bottom',
            plot_caption=element_text(color='grey',size=9,hjust=0),
            plot_caption_position='panel'
            )
    + labs(y = 'Output',
           x='Date',
           title = '',
           caption = f"""Nominal GDP has been seasonally adjusted from the original data set. Since potential output is a function of unemployment 
           due to being calculated through Okun's law, output potential indirectly illustrates the effect of unemployment on the rest of 
           the variables.""")
    + scale_y_continuous(format = 's')
)
ggsave(plot3, 'plot3.png', path= '../images')
plot3

In [89]:
plot = df.copy()
plot['PrimExp'] = plot['PrimExp']/10 + 0.03
plot['Debt'] = plot['Debt']
plot2 = (
    ggplot(data = plot.reset_index(), mapping = aes(x = 'QuarterDate'))
    + geom_line(aes(y = 'OutputGap'), manual_key = 'Output Gap', color = 'red')
    + geom_line(aes(y = 'PrimExp'), manual_key = 'Primary Expenditure')
    + theme_classic()
    + theme(legend_position='top',
            plot_title=element_text(face='bold'),
            axis_text_y = element_blank(),
            plot_subtitle = element_text(color = 'grey'),
            plot_caption = element_text(color='grey')
            )
    + labs(y = 'Normalized Axis', 
           x = '',
           title = 'Primary Expenditure (Measured) and Output Gap (Calculated) Strongly Covary with one Another',
           subtitle = 'Their positive covariance also suggests that Chinese fiscal spending is counter-cyclical.',
           caption = 'Axes are exaggerated for clearer illustration of covariance.'
           )
    + ggsize(1000,500)
    
)
ggsave(plot2, 'plot2.png', path= '../images')
plot2

In [81]:
formula = 'PrimExp ~ DebtMinusHalf + OutputGap'
model1 = smf.ols(formula = formula, data = df).fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:                PrimExp   R-squared:                       0.837
Model:                            OLS   Adj. R-squared:                  0.829
Method:                 Least Squares   F-statistic:                     105.6
Date:                Tue, 04 Mar 2025   Prob (F-statistic):           6.75e-17
Time:                        14:53:47   Log-Likelihood:                 132.46
No. Observations:                  44   AIC:                            -258.9
Df Residuals:                      41   BIC:                            -253.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -0.2627      0.032     -8.137

In [82]:
formula = 'OneYearLPRRate ~ Inflation + OutputGap'
model = smf.ols(formula = formula, data = df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:         OneYearLPRRate   R-squared:                       0.844
Model:                            OLS   Adj. R-squared:                  0.837
Method:                 Least Squares   F-statistic:                     111.1
Date:                Tue, 04 Mar 2025   Prob (F-statistic):           2.81e-17
Time:                        14:53:47   Log-Likelihood:                 186.86
No. Observations:                  44   AIC:                            -367.7
Df Residuals:                      41   BIC:                            -362.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0422      0.001     35.161      0.0

In [98]:
plot = df.copy()
plot['PrimExp'] = plot['PrimExp']/2 + 0.18
plot1 = (
    ggplot(data = plot.reset_index(), mapping = aes(x = 'QuarterDate'))
    + geom_line(aes(y = 'OneYearLPRRate'), manual_key = '1 Year LPR', color = 'blue')
    + geom_line(aes(y = 'PrimExp'), manual_key = 'Primary Expenditure')
    + theme_classic()
    + labs(x = '', 
           y = 'Normalized Axis', 
           title = 'The 1 Year LPR and Primary Expenditure Show High Positive Covariance', 
           subtitle = 'This signifies that Chinese Monetary Policy and Chinese Fiscal Policy do work Hand-in-hand.',
           caption = f'Primary Expenditure and 1 Year LPR have a correlation coefficient of {np.corrcoef(df['PrimExp'],df['OneYearLPRRate'])[0,1]}')
    + theme(plot_title = element_text(face = 'bold'),
            plot_subtitle = element_text(color = 'grey'),
            legend_position='top',
            plot_caption = element_text(color='grey'),
            axis_text_y= element_blank()
            )
    + ggsize(1000,500)
)
ggsave(plot1, 'plot1.png', path= '../images')
plot1